# Assessment of the incident power density on realistic spherical human head model

## TO-DO

- [ ] ~~adjustment of the automatic differentiation on Green's function in free space~~
- [x] generate current for frequencies: [3.5, 6., 26., 30., 60.] GHz
- [ ] for each frequency set the dipole-to-skin distance to: [5, 10, 15, 25, 45] mm
- [ ] divide surface on 21x21 collocation points (Gauss-Legendre points)
- [ ] compute E and H field for the surface area of 1 or 4 cm2
- [ ] compute complex-valued power density distribution

### Summary

For frequencies below 30 GHz compute EM field values and power density over the surface of 4 cm2, for frequencies of 30 GHz and up over the surface of 1 cm2. For 26 GHz, both averaging surfaces should be taken into account.

Simulations should be performed on both spherical (curved) model and planar model. Absolute difference should be plotted against frequencies in range 1 to 100 GHz.